## Import Libraries

In [103]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler , LabelEncoder
from sklearn.model_selection import train_test_split , RandomizedSearchCV , GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import m2cgen
import os
import matplotlib.pyplot as plt
import joblib
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 1000)  # Set max width

Activity Mapper:

0 : walking
1 : jogging
2 : upstairs
3 : downstairs
4 : sitting
5 : standing
6 : sleeping

## Load Data

In [104]:
dataset_path = os.getcwd().replace('/notebook' , '') + "/dataset/"

# Read training data
imu_df1 = pd.read_csv(dataset_path + "imu_data.csv")
imu_df2 = pd.read_csv(dataset_path + "imu_data_v2.csv")
imu_df1 = imu_df1.iloc[:,1:]
imu_df2 = imu_df2.iloc[:,1:]
imu_df2 = imu_df2.drop(columns = (imu_df2.iloc[:,12:16]).columns , axis= 1)
imu_df1.columns = imu_df1.columns.str.replace("'" , "")
imu_df2.columns = imu_df2.columns.str.replace("'" , "")

imu_df = pd.concat([imu_df1, imu_df2] , axis= 0)

In [ ]:
imu_df.columns = imu_df.columns.str.replace("'" , "")
x = imu_df.iloc[:,:-1]
y = imu_df.iloc[:,-1]

In [106]:
x.columns

Index(['attitude.roll', ' attitude.pitch', ' attitude.yaw', ' gravity.x', ' gravity.y', ' gravity.z', ' rotationRate.x', ' rotationRate.y', ' rotationRate.z', ' userAcceleration.x', ' userAcceleration.y', ' userAcceleration.z'], dtype='object')

# Modeling

## Activity Detection

In [107]:
x1, x_test , y1 , y_test = train_test_split(x ,y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

In [108]:
rf = RandomForestClassifier()

param_dist = {
    'n_estimators' : [2 , 3 , 4 , 6 , 8 , 10],
    'max_depth' : [2 , 3 , 4 , 6 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = rf,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
35,0.762725,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 10}"
71,0.758536,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 10}"
70,0.756052,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 8}"
34,0.754571,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 8}"
33,0.754191,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 6}"
69,0.749379,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 6}"
68,0.733850,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 4}"
32,0.732243,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 4}"
31,0.724254,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 3}"
67,0.723953,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 3}"


In [113]:
rf = RandomForestClassifier(n_estimators = 4 , max_depth = 10 , criterion= 'entropy')
rf.fit(x_train , y_train)
rf.score(x_val , y_val)
rf_report = classification_report(rf.predict(x_val) , y_val)
print(rf_report)

              precision    recall  f1-score   support

           0       0.78      0.64      0.70      5179
           1       0.79      0.78      0.79      3796
           2       0.55      0.61      0.58      3029
           3       0.52      0.66      0.58      2586
           4       0.79      0.80      0.80      3627
           5       0.64      0.77      0.70      3045
           6       0.97      0.81      0.88      4401

    accuracy                           0.73     25663
   macro avg       0.72      0.73      0.72     25663
weighted avg       0.75      0.73      0.73     25663



In [114]:
rf_train_report = classification_report(rf.predict(x_train) , y_train)
print(rf_train_report)

              precision    recall  f1-score   support

           0       0.82      0.67      0.74     20877
           1       0.83      0.83      0.83     15006
           2       0.61      0.66      0.63     12424
           3       0.56      0.71      0.63     10252
           4       0.80      0.81      0.80     14603
           5       0.64      0.78      0.70     12123
           6       0.98      0.82      0.89     17364

    accuracy                           0.75    102649
   macro avg       0.75      0.75      0.75    102649
weighted avg       0.77      0.75      0.76    102649



In [115]:
rf_test_report = classification_report(rf.predict(x_test) , y_test)
print(rf_test_report)

              precision    recall  f1-score   support

           0       0.78      0.65      0.71      6466
           1       0.80      0.79      0.79      4758
           2       0.58      0.63      0.60      3846
           3       0.53      0.68      0.59      3209
           4       0.80      0.80      0.80      4551
           5       0.64      0.77      0.70      3807
           6       0.97      0.81      0.88      5441

    accuracy                           0.73     32078
   macro avg       0.73      0.73      0.73     32078
weighted avg       0.75      0.73      0.74     32078



In [116]:
with open(os.getcwd().replace('/notebook' , '') + "/model/activity_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(rf))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/activity_model.py" ))

688362


## Motion Classification

In [117]:
mapper = {
    0 : 1,
    1 : 1,
    2 : 1,
    3 : 1,
    4 : 0,
    5 : 0,
    6 : 0
}

In [118]:
motion_y = y.map(mapper)

x1, x_test , y1 , y_test = train_test_split(x ,motion_y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [119]:
rf = RandomForestClassifier()

param_dist = {
    'n_estimators' : [2 , 3 , 4 , 6 , 8 , 10],
    'max_depth' : [2 , 3 , 4 , 6 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = rf,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
70,0.970014,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 8}"
35,0.969683,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 10}"
71,0.969225,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 10}"
34,0.969147,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 8}"
69,0.968163,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 6}"
33,0.968163,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 6}"
68,0.967442,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 4}"
32,0.966614,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 4}"
65,0.966079,"{'criterion': 'entropy', 'max_depth': 8, 'n_estimators': 10}"
28,0.965952,"{'criterion': 'gini', 'max_depth': 8, 'n_estimators': 8}"


In [120]:
rf = RandomForestClassifier(n_estimators = 4 , max_depth = 10 , criterion= 'entropy')
rf.fit(x_train , y_train)
rf.score(x_val , y_val)
rf_report = classification_report(rf.predict(x_val) , y_val)
print(rf_report)

              precision    recall  f1-score   support

           0       0.96      0.97      0.96     10881
           1       0.98      0.97      0.97     14782

    accuracy                           0.97     25663
   macro avg       0.97      0.97      0.97     25663
weighted avg       0.97      0.97      0.97     25663



In [121]:
rf_train_report = classification_report(rf.predict(x_train) , y_train)
print(rf_train_report)

              precision    recall  f1-score   support

           0       0.96      0.97      0.97     43476
           1       0.98      0.97      0.98     59173

    accuracy                           0.97    102649
   macro avg       0.97      0.97      0.97    102649
weighted avg       0.97      0.97      0.97    102649



In [122]:
rf_test_report = classification_report(rf.predict(x_test) , y_test)
print(rf_test_report)

              precision    recall  f1-score   support

           0       0.96      0.97      0.96     13598
           1       0.98      0.97      0.97     18480

    accuracy                           0.97     32078
   macro avg       0.97      0.97      0.97     32078
weighted avg       0.97      0.97      0.97     32078



In [123]:
with open(os.getcwd().replace('/notebook' , '') + "/model/motion_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(rf))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/motion_model.py" ))

285784


## Steady Movement Classification

In [161]:
steady_df  =imu_df[(imu_df['Target'] == 4) | (imu_df['Target'] == 5) | (imu_df['Target'] == 6)]
x = steady_df.iloc[:,:-1]
y = steady_df.iloc[:,-1]

x1, x_test , y1 , y_test = train_test_split(x , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [125]:
rf = RandomForestClassifier()

param_dist = {
    'n_estimators' : [3 , 4 , 5 , 6 , 8 , 10],
    'max_depth' : [3 , 4 , 5 , 6 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = rf,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
71,0.918662,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 10}"
35,0.918163,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 10}"
69,0.914712,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 6}"
70,0.914690,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 8}"
34,0.913668,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 8}"
33,0.911625,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 6}"
68,0.908855,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 5}"
32,0.905496,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 5}"
67,0.903498,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 4}"
31,0.903089,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 4}"


In [166]:
rf = RandomForestClassifier(n_estimators = 5 , max_depth = 10 , criterion= 'entropy')
rf.fit(x_train , y_train)
rf.score(x_val , y_val)
rf_report = classification_report(rf.predict(x_val) , y_val)
print(rf_report)

              precision    recall  f1-score   support

           4       0.89      0.89      0.89      3713
           5       0.87      0.89      0.88      3596
           6       0.98      0.96      0.97      3704

    accuracy                           0.91     11013
   macro avg       0.91      0.91      0.91     11013
weighted avg       0.91      0.91      0.91     11013



In [167]:
rf_train_report = classification_report(rf.predict(x_train) , y_train)
print(rf_train_report)

              precision    recall  f1-score   support

           4       0.92      0.92      0.92     14770
           5       0.91      0.92      0.91     14490
           6       0.99      0.97      0.98     14791

    accuracy                           0.94     44051
   macro avg       0.94      0.94      0.94     44051
weighted avg       0.94      0.94      0.94     44051



In [168]:
rf_test_report = classification_report(rf.predict(x_test) , y_test)
print(rf_test_report)

              precision    recall  f1-score   support

           4       0.89      0.88      0.88      4635
           5       0.87      0.89      0.88      4512
           6       0.98      0.97      0.97      4619

    accuracy                           0.91     13766
   macro avg       0.91      0.91      0.91     13766
weighted avg       0.91      0.91      0.91     13766



In [169]:
with open(os.getcwd().replace('/notebook' , '') + "/model/steady_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(rf))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/steady_model.py" ))

454630


## Unsteady Movement Classification

In [136]:
unsteady_df  =imu_df[(imu_df['Target'] != 4) & (imu_df['Target'] != 5) & (imu_df['Target'] != 6)]

x = unsteady_df.iloc[:,:-1]
y = unsteady_df.iloc[:,-1]

## Unsteady Motion Classification

In [137]:
"""
Map
0 - surface motion,
1 - staircase motion
"""
mapper = {
    0 : 0,
    1 : 0,
    2 : 1,
    3 : 1
}

In [138]:
unsteady_motion_y = y.map(mapper)

x1, x_test , y1 , y_test = train_test_split(x ,unsteady_motion_y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [139]:
rf = RandomForestClassifier()

param_dist = {
    'n_estimators' : [4 , 5 , 6 , 7 , 8 , 10],
    'max_depth' : [4 , 5 , 6 , 7 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = rf,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
35,0.863818,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 10}"
34,0.863613,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 8}"
71,0.863101,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 10}"
70,0.861207,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 8}"
33,0.860712,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 7}"
32,0.859722,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 6}"
31,0.859483,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 5}"
69,0.859295,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 7}"
67,0.858203,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 5}"
68,0.858118,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 6}"


In [144]:
rf = RandomForestClassifier(n_estimators = 4 , max_depth = 10 , criterion= 'entropy')
rf.fit(x_train , y_train)
rf.score(x_val , y_val)
rf_report = classification_report(rf.predict(x_val) , y_val)
print(rf_report)

              precision    recall  f1-score   support

           0       0.88      0.85      0.87      8343
           1       0.81      0.85      0.83      6307

    accuracy                           0.85     14650
   macro avg       0.85      0.85      0.85     14650
weighted avg       0.85      0.85      0.85     14650



In [145]:
rf_train_report = classification_report(rf.predict(x_train) , y_train)
print(rf_train_report)

              precision    recall  f1-score   support

           0       0.89      0.87      0.88     32965
           1       0.84      0.87      0.85     25633

    accuracy                           0.87     58598
   macro avg       0.87      0.87      0.87     58598
weighted avg       0.87      0.87      0.87     58598



In [146]:
rf_test_report = classification_report(rf.predict(x_test) , y_test)
print(rf_test_report)

              precision    recall  f1-score   support

           0       0.88      0.85      0.87     10417
           1       0.81      0.85      0.83      7895

    accuracy                           0.85     18312
   macro avg       0.85      0.85      0.85     18312
weighted avg       0.85      0.85      0.85     18312



In [147]:
with open(os.getcwd().replace('/notebook' , '') + "/model/unsteady_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(rf))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/unsteady_model.py" ))

334299


## Staircase motion

In [148]:
staircase_df  =imu_df[(imu_df['Target'] == 1) | (imu_df['Target'] == 2)]
x = staircase_df.iloc[:,:-1]
y = staircase_df.iloc[:,-1]

x1, x_test , y1 , y_test = train_test_split(x, y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [149]:
rf = RandomForestClassifier()

param_dist = {
    'n_estimators' : [4 , 5 , 6 , 7 , 8 , 10],
    'max_depth' : [4 , 5 , 6 , 7 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = rf,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
35,0.931659,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 10}"
71,0.931589,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 10}"
34,0.930778,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 8}"
70,0.930390,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 8}"
33,0.929720,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 7}"
69,0.929015,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 7}"
32,0.928310,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 6}"
68,0.926017,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 6}"
31,0.925877,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 5}"
67,0.924677,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 5}"


In [150]:
rf = RandomForestClassifier(n_estimators = 4, max_depth = 10 , criterion= 'entropy')
rf.fit(x_train , y_train)
rf.score(x_val , y_val)
rf_report = classification_report(rf.predict(x_val) , y_val)
print(rf_report)

              precision    recall  f1-score   support

           1       0.93      0.92      0.93      3822
           2       0.91      0.93      0.92      3268

    accuracy                           0.92      7090
   macro avg       0.92      0.92      0.92      7090
weighted avg       0.92      0.92      0.92      7090



In [151]:
rf_train_report = classification_report(rf.predict(x_train) , y_train)
print(rf_train_report)

              precision    recall  f1-score   support

           1       0.95      0.95      0.95     15083
           2       0.94      0.95      0.94     13275

    accuracy                           0.95     28358
   macro avg       0.95      0.95      0.95     28358
weighted avg       0.95      0.95      0.95     28358



In [152]:
rf_test_report = classification_report(rf.predict(x_test) , y_test)
print(rf_test_report)

              precision    recall  f1-score   support

           1       0.93      0.92      0.93      4741
           2       0.91      0.92      0.91      4121

    accuracy                           0.92      8862
   macro avg       0.92      0.92      0.92      8862
weighted avg       0.92      0.92      0.92      8862



In [153]:
with open(os.getcwd().replace('/notebook' , '') + "/model/staircase_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(rf))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/staircase_model.py" ))

288910


## Surface motion classification

In [154]:
surface_df  =imu_df[(imu_df['Target'] == 0) | (imu_df['Target'] == 4)]
x = surface_df.iloc[:,:-1]
y = surface_df.iloc[:,-1]

x1, x_test , y1 , y_test = train_test_split(x , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [155]:
rf = RandomForestClassifier()

param_dist = {
    'n_estimators' : [4 , 5 , 6 , 7 , 8 , 10],
    'max_depth' : [4 , 5 , 6 , 7 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = rf,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
70,0.977616,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 8}"
35,0.977270,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 10}"
71,0.976924,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 10}"
69,0.976547,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 7}"
33,0.976453,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 7}"
34,0.976295,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 8}"
31,0.976075,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 5}"
68,0.976012,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 6}"
67,0.975918,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 5}"
32,0.975792,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 6}"


In [156]:
rf = RandomForestClassifier(n_estimators = 5, max_depth = 10 , criterion= 'gini')
rf.fit(x_train , y_train)
rf.score(x_val , y_val)
rf_report = classification_report(rf.predict(x_val) , y_val)
print(rf_report)

              precision    recall  f1-score   support

           0       0.98      0.97      0.98      4314
           4       0.97      0.98      0.97      3638

    accuracy                           0.98      7952
   macro avg       0.97      0.98      0.98      7952
weighted avg       0.98      0.98      0.98      7952



In [157]:
rf_train_report = classification_report(rf.predict(x_train) , y_train)
print(rf_train_report)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     17086
           4       0.98      0.98      0.98     14722

    accuracy                           0.98     31808
   macro avg       0.98      0.98      0.98     31808
weighted avg       0.98      0.98      0.98     31808



In [158]:
rf_test_report = classification_report(rf.predict(x_test) , y_test)
print(rf_test_report)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      5342
           4       0.97      0.97      0.97      4598

    accuracy                           0.98      9940
   macro avg       0.98      0.98      0.98      9940
weighted avg       0.98      0.98      0.98      9940



In [159]:
with open(os.getcwd().replace('/notebook' , '') + "/model/surface_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(rf))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/surface_model.py" ))

215059
